In [2]:
import pandas as pd
import re
from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [61]:
pd.options.display.max_colwidth = None 

In [3]:
header_list = ['DATE', 'USER', 'RETRY', 'TIME', 'RETURNCODE', 'COMMAND']
data = pd.read_csv('../data/slurm_wrapper_ce6.log', names=header_list, sep=' - ')

data

,DATE,USER,RETRY,TIME,RETURNCODE,COMMAND
0,2020-10-16 10:37:44.163454,user 9202,retry 0,time 0.08495402336120605,returncode 0,"command ['/usr/bin/scontrol', 'show', 'job', '..."
1,2020-10-16 10:37:44.206654,user 9202,retry 0,time 0.08943057060241699,returncode 0,"command ['/usr/bin/scontrol', 'show', 'job', '..."
2,2020-10-16 10:37:44.218760,user 9202,retry 0,time 0.05928945541381836,returncode 0,"command ['/usr/bin/scontrol', 'show', 'job', '..."
3,2020-10-16 10:37:44.256403,user 9202,retry 0,time 0.038695573806762695,returncode 0,"command ['/usr/bin/scontrol', 'show', 'job', '..."
4,2020-10-16 10:37:44.611603,user 9202,retry 0,time 0.03343677520751953,returncode 0,"command ['/usr/bin/scontrol', 'show', 'job', '..."
...,...,...,...,...,...,...
4776515,2021-10-07 21:59:35.014602,user 9221,retry 0,time 0.060086965560913086,returncode 0,"command ['/usr/bin/squeue', '-o', '%i %T', '-u..."
4776516,2021-10-07 21:59:35.238970,user 9202,retry 0,time 0.09804415702819824,returncode 0,"command ['/usr/bin/squeue', '-o', '%i %T', '-u..."
4776517,2021-10-07 21:59:57.265189,user 9203,retry 0,time 0.02454972267150879,returncode 0,"command ['/usr/bin/squeue', '-o', '%i %T', '-u..."
4776518,2021-10-07 22:00:04.024360,user 9201,retry 0,time 0.03941917419433594,returncode 0,"command ['/usr/bin/squeue', '-o', '%i %T', '-u..."


In [4]:
title_remove = {'USER': 'user ', 'RETRY':'retry ', 'TIME':'time ', 'RETURNCODE':'returncode ', 'COMMAND':'command '}

for key in title_remove:
    data[key] = data[key].map(lambda x: x.lstrip(title_remove[key]))

data.head()

,DATE,USER,RETRY,TIME,RETURNCODE,COMMAND
0,2020-10-16 10:37:44.163454,9202,0,0.08495402336120605,0,"['/usr/bin/scontrol', 'show', 'job', '24997187']"
1,2020-10-16 10:37:44.206654,9202,0,0.08943057060241699,0,"['/usr/bin/scontrol', 'show', 'job', '24997190']"
2,2020-10-16 10:37:44.218760,9202,0,0.05928945541381836,0,"['/usr/bin/scontrol', 'show', 'job', '24997193']"
3,2020-10-16 10:37:44.256403,9202,0,0.038695573806762695,0,"['/usr/bin/scontrol', 'show', 'job', '24997189']"
4,2020-10-16 10:37:44.611603,9202,0,0.03343677520751953,0,"['/usr/bin/scontrol', 'show', 'job', '24997192']"


In [5]:
#data = data.drop(columns='RETRY')
#data = data[data['RETURNCODE'] != 'returncode 0']
#data

In [6]:
data_user = data[data['USER'] == '9204']
data_user.head()


,DATE,USER,RETRY,TIME,RETURNCODE,COMMAND
13,2020-10-16 10:38:29.869156,9204,0,0.06946611404418945,0,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x5..."
20,2020-10-16 10:39:44.355935,9204,0,8.835923194885254,0,"['/usr/bin/scontrol', 'show', 'job']"
36,2020-10-16 10:40:51.756875,9204,0,6.003079652786255,0,"['/usr/bin/scontrol', 'show', 'job']"
37,2020-10-16 10:40:55.596886,9204,0,0.14368605613708496,0,"['/usr/bin/sacct', '-j', '24997282', '--noconv..."
307,2020-10-16 11:08:28.127242,9204,0,2.43306303024292,0,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x5..."


In [7]:
#data_user = (data_user[data_user['COMMAND'].str.contains('sbatch')]
#                   .reset_index()
#                   .drop(columns='index'))


data_user['SERVER'] = 'ce6'
data_user.shape

(154862, 7)

In [8]:
data_user['TIME'] = pd.to_numeric(data_user['TIME'])
#data_user = data_user[(data_user['RETURNCODE'] == '1') & (data_user['TIME'] > 15)]
data_user.shape

(154862, 7)

In [9]:
data_two = pd.read_csv('../data/slurm_wrapper_ce5.log', names=header_list, sep=' - ')

In [10]:
for key in title_remove:
    data_two[key] = data_two[key].map(lambda x: x.lstrip(title_remove[key]))

data_two['SERVER'] = 'ce5'
    
data_two.head()

,DATE,USER,RETRY,TIME,RETURNCODE,COMMAND,SERVER
0,2020-10-16 08:15:39.278699,0,0,0.07347559928894043,0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '20...",ce5
1,2020-10-16 08:18:08.313309,0,0,0.18363237380981445,0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '20...",ce5
2,2020-10-16 08:22:48.128689,0,0,0.07547116279602051,0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '20...",ce5
3,2020-10-16 08:25:13.257408,0,0,0.09484362602233887,0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '20...",ce5
4,2020-10-16 08:31:01.460723,0,0,0.07498788833618164,0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '20...",ce5


In [11]:
data_user_two = data_two[data_two['USER'] == '9204']

data_user_two.shape

(159847, 7)

In [12]:
data_user_two['TIME'] = pd.to_numeric(data_user_two['TIME'])
#data_user_two = data_user_two[(data_user_two['RETURNCODE'] == '1') & (data_user_two['TIME'] > 15)]
data_user_two.shape

(159847, 7)

In [28]:
#data_user_two = (data_user_two[data_user_two['COMMAND'].str.contains('sbatch')]
#                   .reset_index()
#                   .drop(columns='index'))
data_user_two.shape

(159847, 7)

In [29]:
logs = data_user.append(data_user_two).reset_index()
logs['TESTING'] = logs['RETURNCODE'] == '0'
logs['UNRESPONSIVE'] = logs['RETURNCODE'] == '1'
logs = logs.drop(columns='index')
logs

,DATE,USER,RETRY,TIME,RETURNCODE,COMMAND,SERVER,TESTING,UNRESPONSIVE
0,2020-10-16 10:38:29.869156,9204,0,0.069466,0,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x5...",ce6,True,False
1,2020-10-16 10:39:44.355935,9204,0,8.835923,0,"['/usr/bin/scontrol', 'show', 'job']",ce6,True,False
2,2020-10-16 10:40:51.756875,9204,0,6.003080,0,"['/usr/bin/scontrol', 'show', 'job']",ce6,True,False
3,2020-10-16 10:40:55.596886,9204,0,0.143686,0,"['/usr/bin/sacct', '-j', '24997282', '--noconv...",ce6,True,False
4,2020-10-16 11:08:28.127242,9204,0,2.433063,0,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x5...",ce6,True,False
...,...,...,...,...,...,...,...,...,...
314704,2021-10-07 21:55:53.066795,9204,0,0.123221,0,"['/usr/bin/squeue', '-o', '%i %T', '-u', 'lcga...",ce5,True,False
314705,2021-10-07 21:56:53.364604,9204,0,0.025579,0,"['/usr/bin/squeue', '-o', '%i %T', '-u', 'lcga...",ce5,True,False
314706,2021-10-07 21:56:53.447106,9204,0,0.024627,0,"['/usr/bin/squeue', '-o', '%i %T', '-u', 'lcga...",ce5,True,False
314707,2021-10-07 21:56:53.529364,9204,0,0.025440,0,"['/usr/bin/squeue', '-o', '%i %T', '-j', '3292...",ce5,True,False


In [62]:
logs_truncated = logs[['USER', 'SERVER', 'COMMAND']]

logs_truncated = (logs_truncated[logs_truncated['COMMAND'].str.contains('sbatch')])
logs_truncated
                  
logs_truncated['JOBID'] = logs_truncated['COMMAND'].str.extract(r".(\d{8,8}).", expand=True)
logs_truncated

,USER,SERVER,COMMAND,JOBID
0,9204,ce6,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x5572a88e00e0.3390891/bl_9b60bf8b8b0d']",NaN
4,9204,ce6,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x5572a6eeaa60.3390891/bl_5c278c121bf0']",NaN
8,9204,ce6,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x5572a72c7980.3390891/bl_f1442e613736']",NaN
12,9204,ce6,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x5572a6f85f50.3390891/bl_2ef97a42c721']",NaN
19,9204,ce6,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x5572a7772750.3390891/bl_c383c50972f5']",NaN
...,...,...,...,...
314676,9204,ce5,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x555675a3e340.1189023/bl_lATCyz']",NaN
314683,9204,ce5,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x555672c93140.1189023/bl_wnaA6C']",NaN
314689,9204,ce5,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x555675febce0.1189023/bl_1ejnfb']",NaN
314695,9204,ce5,"['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x555672fb3590.1189023/bl_vo5ltR']",NaN
